<div style="text-align: center; font-family: 'Georgia'; font-size: 19px;">
    <h2>Lista 2</h2>
</div>

<div style="text-align;font-family: 'Georgia'; font-size: 10px;">
    <h2>Zanim przejdziemy konkretnie do zadań 4-9 to zaimplementujemy odpowiednie funkcje, które posłużą nam przy ich wykonywaniu. <br> Dodatkowo zadania powinny zostać rozwiązane przy pomocy własnej implementacji metody eliminacji Gaussa z częściowym wyborem <br> elementu podstawowego oraz dla porównania - przy pomocy funkcji w bibliotece
scipy.</h2>
</div>  

In [33]:
import numpy as np
from scipy.linalg import solve, det

Zaczniemy od zaimplementowania funkcji przydatnej przy metodzie eliminacji Gaussa, która zamienia miejscami wybrane przez nas wiersze macierzy. Argumentami funkcji będzie nasza macierz oraz indeksy wierszy, które chcemy zamienić miejscami. Następnie po zliczeniu liczby kolum, iterujemy po każdej kolumnie jednego z wierszy i zamienimy z odpowiadającymi im wartościami z drugiego wybranego przez nas wiersza.

In [3]:
def zamiana_wierszy(matrix, i, j):
    #określamy liczbę kolumn macierzy i sprawdzamy czy macierz została prawidłowo zapisana i podana jako argument
    liczba_kolumn = len(matrix[0])
    for row in matrix:
        if len(row) != liczba_kolumn:
            raise ValueError("Błąd: Wszystkie wiersze w macierzy muszą mieć taką samą liczbę kolumn.")
    n = liczba_kolumn
    for k in range(n):
        # Przechowujemy wartość z i-tego wiersza i-tej kolumny w zmiennej tymczasowej 't'
        t = matrix[i][k]
        # Przypisujemy do i-tego wiersza wartość z j-tego wiersza w tej samej kolumnie
        matrix[i][k] = matrix[j][k]
         # Przypisujemy do j-tego wiersza poprzednią wartość z i-tego wiersza (przechowaną w 't')
        matrix[j][k] = t

Możemy teraz zaimplementować funkcję, która wykona eliminacje gaussa na podanej przez nas macierzy, przekształcając ją do postaci schodkowej. Funkcja ta działa na zasadzie przekształcania macierzy przy użyciu operacji na wierszach, aby uzyskać zera poniżej głównych elementów (pivotów) i przygotować macierz do dalszego rozwiązywania układu równań.(wykorzystamy w niej powyżej zaimplemnentowaną funkcję do zamiany wierszy miejscami) . Poniżej znajduje się kod wraz z komentrzami objaśniającymi poszczególne kwestie:

In [11]:
def eliminacja_gaussa(matrix):
    liczba_zamienionych_wierszy = 0                        
    matrix = np.array(matrix, dtype=float)       
    m = len(matrix)                           # Liczba wierszy macierzy
    n = len(matrix[0])                        # Liczba kolumn macierzy
    k = 0                                  # Indeks aktualnej kolumny
    w = 0                                  # Indeks aktualngo wiersza
    
    while w < m and k < n:
        # Znajdujemy wiersz z największym elementem w kolumnie k 
        i_max = np.argmax(np.abs(matrix[w:, k])) + w
        
        # Jeśli największy element jest zerem, przechodzimy do następnej kolumny
        if matrix[i_max, k] == 0:
            k += 1
        else:
            # Jeśli wiersz z największym elementem nie jest aktualnym wierszem, zamień wiersze
            if i_max != w:
                zamiana_wierszy(matrix, w, i_max)
                liczba_zamienionych_wierszy += 1               
            
            # Przeprowadamy teraz eliminację Gaussa, aby uzyskać zera poniżej pivota
            for i in range(w+1, m):
                f = matrix[i, k] / matrix[w, k]     
                matrix[i, k] = 0                 
                for j in range(k+1, n):
                    matrix[i, j] = matrix[i, j] - matrix[w, j] * f   
            
            # Przejdź do następnego wiersza i kolumny (następny pivot)
            w += 1
            k += 1
    
    return matrix, liczba_zamienionych_wierszy             


Gdy mamy już dwie powyższe funkcje, jedną do zamiany wierszy miejscami, a drugą do wykonania eliminacji gaussa, to możemy zaimplementować funkcję, która wykonuje rozwiązanie układu równań liniowych zapisanego w postaci macierzy schodkowej (wynikowej macierzy po eliminacji Gaussa) za pomocą podstawiania wstecznego.

In [17]:
def rozwiazanie_układu_równań_liniowych(matrix):
    m = len(matrix)
    wyniki = np.zeros(m)
    
    # Pętla wykonująca podstawianie wsteczne 
    for i in range(m-1, -1, -1):
        # Przypisujemy wyraz wolny z macierzy do wyniku dla zmiennej x_i
        wyniki[i] = matrix[i][m]
        
        # Pętla do odejmowania wartości znanych już zmiennych z wcześniejszych kroków
        for j in range(i+1, m):
            # Odejmujemy iloczyn elementu macierzy i znanego wyniku z poprzedniego kroku
            wyniki[i] -= matrix[i][j] * wyniki[j]
        
        # Dzielimy przez współczynnik przy x_i, aby otrzymać ostateczny wynik dla zmiennej x_i
        wyniki[i] = wyniki[i] / matrix[i][i]
    return wyniki
 


Kolejną przydatną funkcją, którą wykorzystamy w rozwiązaniu zadań będzie funkcja obliczająca wyznacznik macierzy trójkątnej. 

In [7]:
def wyznacznik_macierzy_trójkątnej(matrix, liczba_zamienionych_wierszy):
    n = len(matrix)
    wynik = 1
    for i in range(n):
        wynik *= matrix[i][i]

    # Jeśli liczba zamian wierszy była nieparzysta, wyznacznik zmienia znak
    # Mnożymy wynik przez (-1) podniesione do potęgi zamiana_wierszy
    # (jeśli zamiana_wierszy jest parzysta, wynik pozostaje taki sam; jeśli nieparzysta, zmienia znak)
    return wynik * (-1) ** liczba_zamienionych_wierszy

Do wykonania zadań zostało nam zaimplementować jeszcze jedną funkcje, która wykonuje eliminację Gaussa-Jordana na macierzy. Ta metoda służy do przekształcenia macierzy do postaci zredukowanej, w której zarówno elementy poniżej, jak i powyżej głównej przekątnej są zerami. Wynikowa macierz umożliwia bezpośrednie odczytanie rozwiązań układu równań liniowych.

In [8]:
def metoda_eliminacji_gaussa_jordana(matrix):
    liczba_zamienionych_wierszy = 0
    matrix = np.array(matrix, dtype=float)
    m = len(matrix)        
    n = len(matrix[0])     
    w = 0
    k = 0

    # Pierwsza część: eliminacja Gaussa (zerowanie elementów poniżej pivota)
    while w < m and k < n:
        i_max = np.argmax(np.abs(matrix[w:, k])) + w

        if matrix[i_max, k] == 0:
            k += 1
        else:
            if i_max != w:
                zamiana_wierszy(matrix, w, i_max)
                liczba_zamienionych_wierszy += 1  
            for i in range(w+1, m):
                f = matrix[i, k] / matrix[w, k]
                matrix[i, k] = 0
                for j in range(k+1, n):
                    matrix[i, j] = matrix[i, j] - matrix[w, j] * f
            w += 1
            k += 1

    # Druga część: eliminacja Jordana (zerowanie elementów powyżej pivota)
    for i in range(m-1, -1, -1):
        # Znajdujemy pozycje głównego elementu (pivota) w wierszu i
        j = np.argmax(np.abs(matrix[i]))

        # Normalizacja pivota, aby wartość w matrix[i][j] wynosiła 1
        matrix[i] /= matrix[i][j]

        # Zerujemy elementów powyżej pivota w kolumnie j
        for k in range(i-1, -1, -1):
            # Odejmowanie odpowiednio przemnożonego wiersza i od wiersza k, aby uzyskać zero w kolumnie j
            matrix[k] -= matrix[i] * matrix[k][j]

    return matrix, liczba_zamienionych_wierszy


<div style="text-align;font-family: 'Georgia'; font-size: 10px;">
    <h2>Zadanie 4</h2>
</div>  

W zadaniu 4 należy rozwiązać układ równań $$ {A} \vec{x} = \vec{b} $$

In [12]:
A = np.array([
    [0, 0, 2, 1, 2],
    [0, 1, 0, 2, -1],
    [1, 2, 0, -2, 0],
    [0, 0, 0, -1, 1],
    [0, 1, -1, 1, -1]
])

b = np.array([1, 1, -4, -2, -1])

rozszerzona_macierz = np.column_stack((A, b))

Do rozwiązania naszego układy skorzystamy z metody eliminacji Gaussa. Posłużymy się wcześniej zaimplementowanym kodem:

In [18]:
print(rozwiazanie_układu_równań_liniowych(eliminacja_gaussa(rozszerzona_macierz)[0]))

[ 2. -2.  1.  1. -1.]


Następnie dla porównania skorzystamy w celu rozwiązania tego układu równań z funkcji solve z modułu scipy.linalg z biblioteki SciPy. 


In [19]:
b2 = np.array([[1], [1], [-4], [-2], [-1]])

In [20]:
print(solve(A,b2))

[[ 2.]
 [-2.]
 [ 1.]
 [ 1.]
 [-1.]]


Jak widać oba wyniki wyszły identycznie, co może wskazywać, że nasza funkcja do eliminacji gaussa została prawidłowo zaimplementowana.

<div style="text-align;font-family: 'Georgia'; font-size: 10px;">
    <h2>Zadanie 5</h2>
</div>  

W zadaniu 5 należy wyznaczyć współczynniki wielomianu: $$y = a_0 + a_1 x + a_2 x^2 + a_3 x^3 + a_4 x^4$$


który przechodzi przez punkty (0, −1), (1, 1), (3, 3), (5, 2) i (6, −2).

In [22]:
A = np.array([
    [1, 0, 0, 0, 0],
    [1, 1, 1, 1, 1],
    [1, 3, 9, 27, 81],
    [1, 5, 25, 125, 625],
    [1, 6, 36, 216, 1296]
])

b = np.array([-1, 1, 3, 2, -2])
rozszerzona_macierz = np.column_stack((A, b))

Macierz 𝐴 w kodzie jest stworzona tak, aby odpowiadała równaniom wynikającym z podstawienia wartości x do kolejnych potęg w wielomianie. 

In [23]:
print(rozwiazanie_układu_równań_liniowych(eliminacja_gaussa(rozszerzona_macierz)[0]))

[-1.          2.68333333 -0.875       0.21666667 -0.025     ]


In [24]:
b2 = np.array([[-1], [1], [3], [2], [-2]])
print(solve(A,b2))

[[-1.        ]
 [ 2.68333333]
 [-0.875     ]
 [ 0.21666667]
 [-0.025     ]]


<div style="text-align;font-family: 'Georgia'; font-size: 10px;">
    <h2>Zadanie 6</h2>
</div>  

W zadaniu 6 należy rozwiązać układ równań $$ {A} \vec{x} = \vec{b} $$

In [29]:
A = np.array([
    [3.5, 2.77, -0.76, 1.8],
    [-1.8, 2.68, 3.44, -0.09],
    [0.27, 5.07, 6.9, 1.61],
    [1.71, 5.45, 2.68, 1.71],
], dtype=float)

b = np.array([7.31, 4.23, 13.85, 11.55])

rozszerzona_macierz = np.column_stack((A, b))

b2 = np.array([[7.31], [4.23], [13.85], [11.55]])

x = rozwiazanie_układu_równań_liniowych(eliminacja_gaussa(rozszerzona_macierz)[0])
print(x)

[1. 1. 1. 1.]


In [30]:
print(solve(A,b2))

[[1.]
 [1.]
 [1.]
 [1.]]


Następnie należy wyznaczyć:$$ detA \, \, \, i \, \, \, A\vec{x}$$ 


In [27]:
print(f"det(A) = {wyznacznik_macierzy_trójkątnej(eliminacja_gaussa(A)[0], eliminacja_gaussa(A)[1])}")

det(A) = -0.22579734000000884


In [34]:
det_A = det(A)
print("Wyznacznik macierzy A:", det_A)

Wyznacznik macierzy A: -0.22579733999999502


In [32]:
Ax = np.dot(A, x)
print(f"Ax = {Ax}")

reszta = b - Ax
print(f"r = {reszta}")

Ax = [ 7.31  4.23 13.85 11.55]
r = [8.88178420e-16 0.00000000e+00 1.77635684e-15 1.77635684e-15]


Nasz wektor reszt wskazuje, że wyniki obliczone są z dużą dokładnościa, a błąd jest mały (rzędu $10^{-15}$ i $10^{-16}$). Najbardziej dokładnie okazuje się rowiązanie $x_{2}$.

<div style="text-align;font-family: 'Georgia'; font-size: 10px;">
    <h2>Zadanie 7</h2>
</div>  

W zadaniu 7 ponownie należy rowiązać układ równań.

In [36]:
A = np.array([
    [10, -2, -1, 2, 3, 1, -4, 7],
    [5, 11, 3, 10, -3, 3, 3, -4],
    [7, 12, 1, 5, 3, -12, 2, 3],
    [8, 7, -2, 1, 3, 2, 2, 4],
    [2, -15, -1, 1, 4, -1, 8, 3],
    [4, 2, 9, 1, 12, -1, 4, 1],
    [-1, 4, -7, -1, 1, 1, -1, -3],
    [-1, 3, 4, 1, 3, -4, 7, 6]
], dtype=float)

b = np.array([0, 12, -5, 3, -25, -26, 9, -7])

rozszerzona_macierz = np.column_stack((A, b))

b2 = np.array([[0], [12], [-5], [3], [-25], [-26], [9], [-7]])

x = rozwiazanie_układu_równań_liniowych(eliminacja_gaussa(rozszerzona_macierz)[0])
print(x)

print(solve(A,b2))

[-1.  1. -1.  1. -1.  1. -1.  1.]
[[-1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [ 1.]]


Wynik przy pomocy zaimplementowanej własnoręcznie metody i przy pomocy funkcji z biblioteki Scipy wyszedł ponownie zgodny.

<div style="text-align;font-family: 'Georgia'; font-size: 10px;">
    <h2>Zadanie 8</h2>
</div>  

W zadaniu 8 należy znaleźć macierz odwrotną.

In [37]:
A = np.array([
    [2, -1, 0, 0, 0, 0],
    [-1, 2, -1, 0, 0, 0],
    [0, -1, 2, -1, 0, 0],
    [0, 0, -1, 2, -1, 0],
    [0, 0, 0, -1, 2, -1],
    [0, 0, 0, 0, -1, 5]
], dtype=float)

In [41]:
def macierz_odwrotna(mat):
    mat = np.array(mat, dtype=float)
    m = len(mat)  
    n = len(mat[0])  

    # Sprawdzenie, czy macierz jest kwadratowa 
    if m != n:
        raise(ValueError("Macierz powinna byc kwadratowa"))

    # Sprawdzenie, czy macierz jest osobliwa 
    if np.isclose(wyznacznik_macierzy_trójkątnej(eliminacja_gaussa(mat)[0], eliminacja_gaussa(mat)[1]), 0):
        raise(ValueError("Macierz 0sobliwa, nie ma odwrotności"))

    h = 0  
    mat_inv = np.eye(m)  # Tworzenie macierzy jednostkowej o tym samym rozmiarze co mat

    while h < m:
        i_max = np.argmax(np.abs(mat[h:, h])) + h
        if mat[i_max, h] == 0:
            raise(ValueError("Macierz osobliwa, nie ma odwrotności"))
        else:
            if i_max != h:
                zamiana_wierszy(mat, h, i_max)  
                zamiana_wierszy(mat_inv, h, i_max) 

            for i in range(h + 1, m):  
                f = mat[i, h] / mat[h, h] 
                mat[i, h] = 0
                mat_inv[i] -= mat_inv[h] * f  
                for j in range(h + 1, n): 
                    mat[i, j] -= mat[h, j] * f
            h += 1  

    #pętla do normalizacji wierszy i uzyskania macierzy odwrotnej
    for i in range(n - 1, -1, -1):
        mat_inv[i] /= mat[i, i]  # Dzielimy każdy element wiersza przez element główny
        mat[i] /= mat[i, i]  # Normalizujemy wiersz w oryginalnej macierzy
        for k in range(i):  # Dla wszystkich wierszy powyżej wiersza i
            mat_inv[k] -= mat_inv[i] * mat[k, i]  # Modyfikowanie macierzy odwrotnej
            mat[k] -= mat[i] * mat[k, i]  # Modyfikowanie oryginalnej macierzy

    return mat_inv  


In [43]:
print((macierz_odwrotna(A)))

#porównanie wyniku przy pomocy funkcji inv z modułu linalg
print(np.linalg.inv(A))

[[0.84 0.68 0.52 0.36 0.2  0.04]
 [0.68 1.36 1.04 0.72 0.4  0.08]
 [0.52 1.04 1.56 1.08 0.6  0.12]
 [0.36 0.72 1.08 1.44 0.8  0.16]
 [0.2  0.4  0.6  0.8  1.   0.2 ]
 [0.04 0.08 0.12 0.16 0.2  0.24]]
[[0.84 0.68 0.52 0.36 0.2  0.04]
 [0.68 1.36 1.04 0.72 0.4  0.08]
 [0.52 1.04 1.56 1.08 0.6  0.12]
 [0.36 0.72 1.08 1.44 0.8  0.16]
 [0.2  0.4  0.6  0.8  1.   0.2 ]
 [0.04 0.08 0.12 0.16 0.2  0.24]]


Macierz $A^{-1}$ nie jest trójdiagonalna, ponieważ macierz trójdiagonalna jest macierzą, której wszystkie współczynniki, które nie znajdują się ani na głównej przekątnej, ani na przekątnej tuż powyżej, ani na przekątnej tuż poniżej, wynoszą zero. W naszym przypadku tak nie ma. 

In [45]:
def czy_trójdiagonalna(matrix):
    # Sprawdzenie, czy macierz jest kwadratowa
    rows, cols = matrix.shape
    if rows != cols:
        return False
    
    # Sprawdzenie elementów poza główną przekątną oraz pierwszą nad i pod główną
    for i in range(rows):
        for j in range(cols):
            # Sprawdzenie, czy elementy poza trójdiagonalnymi przekątnymi są zerowe
            if abs(i - j) > 1 and matrix[i, j] != 0:
                return False
    return True

print(czy_trójdiagonalna((macierz_odwrotna(A))))

False


<div style="text-align;font-family: 'Georgia'; font-size: 10px;">
    <h2>Zadanie 9</h2>
</div>  

W zadaniu 9 również musimy znaleźć macierz odwrotną.

In [52]:
A = np.array([
    [1, 3, -9, 6, 4],
    [2, -1, 6, 7, 1],
    [3, 2, -3, 15, 5],
    [8, -1, 1, 4, 2],
    [11, 1, -2, 18, 7]
], dtype=float)

In [47]:
print((macierz_odwrotna(A)))

ValueError: Macierz 0sobliwa, nie ma odwrotności

In [48]:
print(np.linalg.inv(A))

[[-1.06340236e+15 -1.06340236e+15 -1.06340236e+15 -2.12680472e+15
   2.12680472e+15]
 [-5.05116121e+15 -5.05116121e+15 -5.05116121e+15 -1.01023224e+16
   1.01023224e+16]
 [-8.45888241e+14 -8.45888241e+14 -8.45888241e+14 -1.69177648e+15
   1.69177648e+15]
 [-7.02563008e-17 -9.54097912e-18  1.00000000e+00  1.00000000e+00
  -1.00000000e+00]
 [ 2.15097296e+15  2.15097296e+15  2.15097296e+15  4.30194591e+15
  -4.30194591e+15]]


In [55]:
print(wyznacznik_macierzy_trójkątnej(eliminacja_gaussa(A)[0],eliminacja_gaussa(A)[1]))



5.928590951498336e-14


In [54]:
print(np.linalg.det(A))

4.137662434899883e-14


Wyznacznik macierzy A powinien być równy zero według obliczeń na kartce oraz sprawdzenia obliczeń w internecie na stronach, które potrafią wyliczyć wyznacznik.Przez pewne przybliżenia wykonywane przez komputer nie jest on równy zero, co pokazuje małą dokładność. Liczona zatem jest macierz odwrotna z macierzy osobliwej, która jest nieodwracalna, więc wynik nie jest prawdziwy. W niektórych przypadkach funkcja inv w NumPy może próbować obliczyć tzw. pseudo-odwrotność lub generować wyniki, które są numerycznie bardzo niestabilne. Takie wyniki są zazwyczaj obarczone błędami numerycznymi i nie można ich traktować jako rzeczywistej odwrotności.